In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#using requests and bs4
#today: https://www.worldometers.info/coronavirus/?utm_campaign=homeAdvegas1?%22%20%5Cl%22countries
#yesterday: https://www.worldometers.info/coronavirus/#nav-yesterday
url = "https://www.worldometers.info/coronavirus/"
data = requests.get(url)
soup = BeautifulSoup(data.text,'html5lib')

In [3]:
covid_dict = {}
#find all div tags whose id equal to "maincounter-wrap"
divs = soup.find_all("div", {"id": "maincounter-wrap"})
#in each div tag having "maincounter-wrap" id, we find all div tags having "maincounter-number-number" id
for div in divs:
  content_div = div.find("div",{"class":"maincounter-number"})
  #we extract number in span tag, save it to columns named after the text in "h1" tag 
  covid_dict[div.find("h1").text.replace(":","").strip()] = content_div.find("span").text.strip()
print(covid_dict)

{'Coronavirus Cases': '506,080,188', 'Deaths': '6,228,281', 'Recovered': '458,208,175'}


In [4]:
#define all columns label appeared in worldometers
tables = soup.find_all("table")
table_header = tables[0].find_all("th")
table_head = []
for i in range(15):
  if i != 0:
    table_head.append(table_header[i].text.replace("\n","").replace("\xa0",""))
print(table_head)

['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'TotCases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population']


In [5]:
#save to dataframe
covid_data = pd.DataFrame(columns=table_head)
for row in tables[0].tbody.find_all('tr'):
  col = row.find_all('td')
  if (col != []):
    country = col[1].text.strip()
    totalCases = col[2].text.strip()
    newCases = col[3].text.strip()
    totalDeaths = col[4].text.strip()
    newDeaths = col[5].text.strip()
    totalRecovered = col[6].text.strip()
    newRecovered = col[7].text.strip()
    activeCases = col[8].text.strip()
    serious = col[9].text.strip()
    totalCases_per_m = col[10].text.strip()
    deaths = col[11].text.strip()
    totalTests = col[12].text.strip()
    tests_per_m = col[13].text.strip()
    population = col[14].text.strip()
    covid_data = covid_data.append({"Country,Other": country,"TotalCases": totalCases,"NewCases": newCases,
                                    "TotalDeaths": totalDeaths,"NewDeaths": newDeaths,"TotalRecovered": totalRecovered,
                                    "NewRecovered": newRecovered,"ActiveCases": activeCases,"Serious,Critical": serious,
                                    "TotCases/1M pop": totalCases_per_m,"Deaths/1M pop": deaths,"TotalTests": totalTests,
                                    "Tests/1M pop": tests_per_m,"Population": population}, ignore_index=True)
#drop first 7 rows (which are the nunber for each continent)
covid_data.drop(covid_data.index[:7], inplace=True)

In [6]:
#reset index after delete 7 rows, starting from 0
covid_data.reset_index(drop=True)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population
0,World,"506,080,188","+221,799","6,228,281",+429,"458,208,175","+351,993","41,643,732","41,970","64,925",799.0,,,
1,USA,"82,416,687",,"1,016,159",,"80,276,197",,"1,124,331","1,436","246,399","3,038","997,115,318","2,981,044","334,485,290"
2,India,"43,047,594",,"522,006",,"42,513,248",,"12,340",698,"30,653",372,"832,506,755","592,807","1,404,347,026"
3,Brazil,"30,279,270",,"662,266",,"29,298,848",,"318,156","8,318","140,657","3,076","63,776,166","296,261","215,270,453"
4,France,"27,816,299",,"144,387",,"25,301,145",,"2,370,767","1,541","424,465","2,203","260,504,402","3,975,184","65,532,665"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Niue,7,,,,7,,0,,"4,253",,,,"1,646"
225,Nauru,3,,,,3,,0,,274,,,,"10,946"
226,Saint Helena,2,,,,2,,0,,327,,,,"6,109"
227,Micronesia,1,,,,1,,0,,9,,,,"117,186"


In [7]:
#phải chỉnh tên file lưu cho đúng theo từng ngày nha mn
covid_data.to_csv("covid_data_05_24_22.csv",index=False)